In [ ]:
import warnings
warnings.filterwarnings('ignore')
import transect_viz
from transect_viz import transect_generator
from transect_viz import transect_cdec_data
from transect_viz import transect_data
import numpy as np
import holoviews as hv
from holoviews import opts, dim

import pandas as pd
import holoviews as hv
import panel as pn
pn.extension()

# Load transect data

Load the transect file containing lon,lat and values from a .csv file

Then create a line from that transect sampling by connecting them in the order of timestamp and generating a distance along that line starting with the earliest sampled transect to the last one. 

Also read the observation station locations and the distance along the transect to them

In [ ]:
df=transect_data.load_transect_file('../data/Transect_20220302_Fabian.csv')
df['EC'] = df['Sonde1_SpCond']
# get transect and add linear reference distance as 'transect_dist'
gdf = transect_generator.to_geoframe(df)
transect_line=transect_generator.create_transect_line(gdf)
gdf = transect_generator.add_transect_dist(gdf, transect_line)
gdf = gdf.astype({'DateTime':np.datetime64})
# stations also add them to the linear reference
cdec_stations=['ORM', 'ODM', 'OLD', 'GLE', 'GLC']
dfs = transect_cdec_data.get_stations_cached(cdec_stations)
gdfs = transect_generator.to_geoframe(dfs)
gdfs = transect_generator.add_transect_dist(gdfs, transect_line)
#

## Start and end times

The start and end times to be used to build the transects. This is based on a buffered start and end dates around the transect collection time. However this can be any start and end date specified

In [ ]:
bsdate, bedate = transect_viz.get_buffered_start_end_dates(df)
#

## Get CDEC data (EC and Flow)
Specify cache=False else the data will be read from the .pkl file it stores the EC and Flow data once cached

In [ ]:
df_cdec_15 = transect_cdec_data.get_ec_cdec_data(bsdate, bedate, cdec_stations, '15T')
flow_data_15 = transect_cdec_data.get_flow_cdec_data(bsdate, bedate, cdec_stations, '15T')
transect_min, transect_max = transect_generator.get_transect_min_max(gdf)

## Interpolate to fill in missing values

Just inspect the data to ensure there are not too many missing values

In [ ]:
df_cdec_15=df_cdec_15.interpolate()
flow_data_15=flow_data_15.interpolate() # fill nans

## Define function to..

 * Create values for dfec along the transect line defined by gdf
 * create water quality map for the generated transect values from dfec
 * create a vector field for dfflow at the stations defined by gdfs
 * create a vector field map 
 * overlay the water quality and vector field maps

In [ ]:
def create_transect_map(date_value, dfec, gdf, gdfs, value_range):
    dfresult = transect_generator.create_transect(
        date_value, dfec, gdf=gdf, gdfs=gdfs)
    tmap = transect_generator.show_map(dfresult, value_range=value_range).opts(
        title=f'Transect on {date_value}')
    return tmap


def create_vectorfield_map(date_value, dfflow, gdfs, value_range):
    vfmap = transect_viz.create_vector_field_map(transect_viz.create_vector_field(date_value, dfflow, gdfs, mag_column='flow'),
                                                 angle_column='angle', mag_column='flow',
                                                 mag_factor=0.3, line_width=5)
    return vfmap


def show_transect_map(date_value, dfec, dfflow, gdf, gdfs, value_range=None):
    tmap = create_transect_map(
        date_value, dfec, gdf, gdfs, value_range=value_range)
    vfmap = create_vectorfield_map(
        date_value, dfflow, gdfs, value_range=value_range)
    return tmap.Overlay.I * vfmap

# Get Transect Begin and end times

In [ ]:
sdate, edate = transect_viz.get_start_end_dates(gdf)

In [ ]:
def get_interpolated_in_time_transect(dateval, dfec, gdf, gdfs, sampling_freq='15T', value_column='EC'):
    d=pd.to_datetime(dateval)
    gdf2 = dfec.loc[[d.floor(sampling_freq)]]
    dfec1 = dfec.loc[[d.ceil(sampling_freq)]]
    st = transect_generator.create_transect(dfec0, gdf=gdf, gdfs=gdfs)
    et = transect_generator.create_transect(dfec1, dfec, gdf=gdf, gdfs=gdfs)
    td=(d-d.floor(sampling_freq))/pd.Timedelta(sampling_freq)
    return st+(et-st)*td

In [ ]:
def create_transect_from_interpolated_observed_stations_in_time_and_distance(dfec, gdf, gdfs, value_column='EC'):
    gdf2 = gdf.copy()
    gdf2[value_column] = np.nan
    freqstr = dfec.index.freqstr
    for ct in gdf2.DateTime.unique():
        ctransect = get_interpolated_in_time_transect(
            ct, dfec, gdf2, gdfs, sampling_freq=freqstr, value_column=value_column)
        gdf2.loc[gdf2.DateTime == ct,
                 value_column] = ctransect.loc[gdf2.loc[gdf2.DateTime == ct].index]
    return gdf2

In [ ]:
gdf2=gdf.copy()

In [ ]:
gdf2['EC']=df_cdec_15['2022-03-02 15:00']

In [ ]:
df_cdec_15.loc[['2022-02-28 00:00']]

In [ ]:
df_cdec_15.index.freqstr

In [ ]:
gdf2.DateTime.unique()

In [ ]:
gdf2 = create_transect_from_interpolated_observed_stations_in_time_and_distance(df_cdec_15, gdf, gdfs)

In [ ]:
import holoviews as hv
from holoviews import opts, dim

In [ ]:
pts,pts_legend = transect_viz.map_transect_with_size_and_color(gdf)
map_transect = hv.element.tiles.CartoLight()*pts.opts(frame_width=720, colorbar=True, clim=(400,1000))

In [ ]:
pts,pts_legend = transect_viz.map_transect_with_size_and_color(gdf2)
map_transect_generated = hv.element.tiles.CartoLight()*pts.opts(frame_width=720, colorbar=True, clim=(400,1000))

In [ ]:
layout = map_transect.opts(title='Measured Transect') + map_transect_generated.opts(title='Generated Transect from Observation Stations')

In [ ]:
hv.save(layout.cols(1),'Measured_vs_Generated_Transect.html')